In [1]:
import requests_cache
import os
import re
from urllib2 import Request, urlopen
import pandas as pd
import json
import requests
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
def search(key,q):
    ''' Seach the food information from USDA
    Args: key(string) for the password,q(string) for the search name
    
    Returns:response(json) for the search result. If there is not result, return None
    
    '''
    url = 'https://api.themoviedb.org/3'
    response = requests.get(url, params = {'q':q,
                                          '_sort':'r',
                                          'api_key':key,
                                           'format':'json'
                                            })
    try:
        response = response.json()['list']['item']
    except:
        return []
    
    return response

In [21]:
response = requests.get('https://api.themoviedb.org/3/movie/550?api_key=0149893a207211eb2778cdae6c5c06f9&_imdb_id = 1345836')

In [23]:
response.json()

{u'adult': False,
 u'backdrop_path': u'/wSJPjqp2AZWQ6REaqkMuXsCIs64.jpg',
 u'belongs_to_collection': None,
 u'budget': 63000000,
 u'genres': [{u'id': 18, u'name': u'Drama'}],
 u'homepage': u'http://www.foxmovies.com/movies/fight-club',
 u'id': 550,
 u'imdb_id': u'tt0137523',
 u'original_language': u'en',
 u'original_title': u'Fight Club',
 u'overview': u'A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground "fight clubs" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.',
 u'popularity': 13.488462,
 u'poster_path': u'/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg',
 u'production_companies': [{u'id': 508, u'name': u'Regency Enterprises'},
  {u'id': 711, u'name': u'Fox 2000 Pictures'},
  {u'id': 20555, u'name': u'Taurus Film'},
  {u'id': 54050, u'name': u'Linson Films'},
  {u'id': 54051, u'name': u'Atman Entertainment'},
  {u'